In [1]:
from utils.parse_config import *
from models import create_modules
config_path='config/yolov3.cfg'
module_defs=parse_model_config(config_path)
h,model=create_modules(module_defs)

In [2]:
print(h)

{'type': 'net', 'batch': '16', 'subdivisions': '1', 'width': '416', 'height': '416', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}


In [1]:
from senet.se_resnet import *
semodel=se_resnet50()

In [3]:
semodel.layer1

Sequential(
  (0): SEBottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (se): SELayer(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc): Sequential(
        (0): Linear(in_features=256, out_features=16, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=16, out_features=256, bias=False)
        (3): Sigmoid()
      )
    )
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2

In [4]:
semodel.layer2

Sequential(
  (0): SEBottleneck(
    (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (se): SELayer(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (fc): Sequential(
        (0): Linear(in_features=512, out_features=32, bias=False)
        (1): ReLU(inplace=True)
        (2): Linear(in_features=32, out_features=512, bias=False)
        (3): Sigmoid()
      )
    )
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): Ba

In [31]:
import numpy as np
from PIL import Image
from PIL import ImageFile
import torchvision
img_path='data\insect\JPEGImages\\test\996.jpeg'
img = np.array(Image.open(img_path).convert('RGB').resize((416,416)), dtype=np.uint8)
img=img.transpose((2,0,1))

In [32]:
img.shape

(3, 416, 416)

In [33]:
import torch
img=torch.Tensor(img).unsqueeze(0)
semodel(img).shape

torch.Size([1, 1000])

In [34]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

In [35]:
semodel.fc=Identity()

In [36]:
semodel(img).shape

torch.Size([1, 2048])

In [37]:
semodel.fc=model[1]

In [39]:
from models import *
darkmodel=Darknet("config/yolov3.cfg")

In [1]:
"""
要在SE net中间插三个yolo
"""
from senet.se_module import *
model=SELayer(3,reduction=3)
import torchvision
import torch
import os
import numpy as np
from PIL import Image
img_path="assets\messi.png"
img=np.array(Image.open(img_path).convert('RGB'), dtype=np.uint8)
img=torchvision.transforms.ToTensor()(img)
img=torchvision.transforms.Resize((416,416))(img)
img=torch.unsqueeze(img,0)
model

SELayer(
  (avg_pool): AdaptiveAvgPool2d(output_size=1)
  (fc): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=False)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=1, out_features=3, bias=False)
    (3): Sigmoid()
  )
)

In [2]:
model(img).size()

torch.Size([1, 3, 416, 416])

In [3]:
from senet.se_resnet import SEBasicBlock
model1=SEBasicBlock(inplanes=3,planes=3,stride=1,reduction=1)

In [4]:
model1

SEBasicBlock(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (se): SELayer(
    (avg_pool): AdaptiveAvgPool2d(output_size=1)
    (fc): Sequential(
      (0): Linear(in_features=3, out_features=3, bias=False)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=3, out_features=3, bias=False)
      (3): Sigmoid()
    )
  )
)

In [5]:
from senet.se_resnet import SEBottleneck
SEBottleneck(3,3,reduction=1)

SEBottleneck(
  (conv1): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(3, 12, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (se): SELayer(
    (avg_pool): AdaptiveAvgPool2d(output_size=1)
    (fc): Sequential(
      (0): Linear(in_features=12, out_features=12, bias=False)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=12, out_features=12, bias=False)
      (3): Sigmoid()
    )
  )
)